In [1]:
import pandas as pd
import numpy as np
import re
#natural language processing 
import nltk
#stop words dont add much meaning to text or paragraph giuven:The, a, what, where
nltk.download('stopwords')
from nltk.corpus import stopwords
#stemming removes the prefix and the suffix of a word and givesd teh root meaning
from nltk.stem.porter import PorterStemmer
#Turns the text into features vectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#Get the news into the dataframe
news_set = pd.read_csv("data/train.csv")
news_set.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shriv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
forbidden = stopwords.words('english')#these are the stop words we need to remove in the future

For this program, we aim to preprocess the data in order to feed it in ot a binary classfication model, logistic regression to decipher whether the news we have at hand is fake or real. 

In [3]:
#chekc whther there is na values in datasets
news_set.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
#Since there are so many missing values, we dont need to do imputations
#We can replace the values will null statements with null string
news_set = news_set.fillna('')

Preproccesing 1, we can use either teh text data alone since it has so much data, or we can use title+ author. Title +author will still gtive a very good accuracy score.

In [5]:
#news_set.columns
#keep the relevant features 
relevant_feats_for_second_processing = ['title']
#merge the title and author columns together
news_set["content"] = news_set["author"]+" "+news_set["title"]

In [6]:
#seperating the data and labels
X = news_set.drop(columns = 'label',axis = 1)
Y = news_set['label']
#X,Y

Stemming :
The process of reducing a word into its root word

In [7]:
#Use porter stemmer
port_stem = PorterStemmer()

def stemming(content):
    #no numbers
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    #make everything lowercase and split into array
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    #go through the array and stem each word
    new_content = []
    for w in stemmed_content:
        if w not in forbidden:
            new_content.append(port_stem.stem(w))
    stemmed_content = ' '.join(new_content)
    return stemmed_content
    

In [8]:
news_set['content'] = news_set['content'].apply(stemming)

In [9]:
#print(news_set['content'])
news_set.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...


In [10]:
#serperating the data and label
#get the trains and test spliut
X = news_set['content'].values
Y = news_set['label'].values


In [11]:
#Ge tthe text into features for the model to work on it
v = TfidfVectorizer()
v.fit(X)
X = v.transform(X)


In [12]:
#Splitting into train and test data
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = .20,stratify=Y, random_state = 2)


In [13]:
m = LogisticRegression()
m.fit(X_train,Y_train)


LogisticRegression()

In [14]:
#accuracy on training daqta
x_train_pred = m.predict(X_train)
tr_dat_acc = accuracy_score(x_train_pred,Y_train)
tr_dat_acc

0.9865985576923076

In [15]:
#test data
X_test_dat = m.predict(X_test)
test_dat_acc = accuracy_score(X_test_dat,Y_test)
test_dat_acc

0.9790865384615385

In [16]:
#creating function where we convert the 1s and 0s into there meaning which is the real or fake
def predictions_of_ind(i):
    X_new = X_test[i]
    prediction = m.predict(X_new)

    if prediction[0].astype(int)>0:
        return "Fake"
    else:
        return "Real"

In [17]:
predictions_of_ind(35)

'Real'

In [18]:
type(predictions_of_ind(5))

str

In [19]:
#creating function for y test values or labels to denote whether they are real or fake
def pred_y_ind(i):
    y_new = Y_test[i]
    if(y_new.astype(int) >0):
        return "Fake"
    else:
        return "Real"

In [20]:
#create a list of the first 34 news sources and see
for i in range(1,34):
    print("Prediction: "+ predictions_of_ind(i) +" Actual: "+ pred_y_ind(i))

Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Fake Actual: Real
Prediction: Fake Actual: Fake
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Fake Actual: Fake
Prediction: Real Actual: Fake
Prediction: Real Actual: Real
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Real Actual: Real
Prediction: Real Actual: Real
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Real Actual: Real
Prediction: Fake Actual: Fake
Prediction: Fake Actual: Fake
